#### PNP - Rule Based Analytics Modelling

The Objective of this project is to use historical fraud analytics data using rule based modelling technique to detect customers whose fraud claims would be auto paid or referred. This would help the fraud team to be able to pay attention to high risk claims.

#### Getting the Data: Creating a fraud data

For this exercies we would be using a synthetic data which would fairly represent the real data, we will using Faker to generate this fake data.

#### Required Libraries

In [1]:
import numpy as np
import pandas as pd
from faker import Faker
from pandas import read_csv
from pandasql import sqldf
import datetime

# create some fake data[]
#locale_list = ['en_UK','en_US']
fake = Faker(['en_UK','en_US'])

#### Generating Fake Data

In [106]:
#Adding duplicates to the customer profile
#create customer profile
#transaction profile
def synthetic_data(cust_num,trans_num):
    import numpy as np
    import pandas as pd
    from pandasql import sqldf
    from faker import Faker
    def customer_profile(cust_num):
        """ The Function takes the number of observation to be generated as argument to create customer data """
        
        # create some fake data[]
        #locale_list = ['en_UK','en_US']
        fake = Faker(['en_UK','en_US'])
        
        # lists to randomly assign to workers
        status_list = ['Full Time', 'Part Time', 'Per Diem','Resigned']
        gender = ['Male','Female']
        account_type = ['Current','Savings']
        #team_list = [fake.color_name() for x in range(5)]
        first_claim = ['Yes','No']

        cust_df = [{'claim_id': x + 500000001,
                    'customer_id':x+119001,
                  #'customer_id':np.random.choice(customer_id), #This help to introduce duplicates
                  'customer_name':fake.name(), 
                  'relationship_start_date':fake.date_between(start_date='-12y', end_date='-1y'),
                  'country':fake.country(),
                  'gender':np.random.choice(gender, p=[0.59,0.41]),
                  #'residence_address':fake.address(),
                  'postcode':fake.postcode(),
                  'employment_status':np.random.choice(status_list, p=[0.45, 0.25, 0.20,0.10]), # assign items from list with different probabilities                
                  'age':np.random.randint(18,95),                
                  'account_type':np.random.choice(account_type, p=[0.55,0.45]),
                  'first_claim':np.random.choice(first_claim, p=[0.55,0.45]),
                  'total_claims_in_90days': np.random.randint(0,10),
                  'no_of_trans':np.random.randint(15),
                  #'team':np.random.choice(team_list)
                   } for x in range(cust_num)]
        
        return cust_df
  
    cust_table = pd.DataFrame(customer_profile(cust_num))
    
    def trans_profile(trans_num):
        """ This function take number of observation to be generated to create transaction data using randomly generated 
        customer ID obtained from function above"""
        
        trans_df = [{ 'customer_id':np.random.choice(cust_table.customer_id),
                  'trans_id':str(x+5180711001),
                  'trans_datetime': fake.date_this_year(),
                  'trans_amount':np.random.randint(1000),
                  #'claim_amount':round(np.random.uniform(20,1000),2),            
                  
        
                } for x in range(trans_num) ]
        return trans_df
    trans_table = pd.DataFrame(trans_profile(trans_num))
    
    """ Left join of the customer and transaction table """
    
    
    df = sqldf("select a.*,b.trans_id,b.trans_datetime,b.trans_amount,c.claim_amount,c.claim_vol from cust_table a left join trans_table b on a.customer_id=b.customer_id \
    inner join (select customer_id, sum(trans_amount) as claim_amount, count(customer_id) as claim_vol from trans_table group by customer_id) c on c.customer_id=b.customer_id ")
    
    return df
    

In [96]:
fake_df = synthetic_data(cust_num=20,trans_num=70)
fake_df.head()

,claim_id,customer_id,customer_name,relationship_start_date,country,gender,postcode,employment_status,age,account_type,first_claim,total_claims_in_90days,no_of_trans,trans_id,trans_datetime,trans_amount,claim_amount,claim_vol
0,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711005,2022-04-23,150,2333,5
1,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711017,2022-01-09,833,2333,5
2,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711040,2022-03-25,39,2333,5
3,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711057,2022-04-14,610,2333,5
4,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711067,2022-01-31,701,2333,5


#### Storing the Data for Future Use

In [99]:
#For Consistency sake i have stored the data as pickle file which means i can always load it wheneven i need the data
# I am thinkig there should be a way to set seed for Faker to return same data everytime but i do not have any answer yet.
#fake_df.to_pickle('pnp_data')

In [100]:
# read pickle file as dataframe
df = pd.read_pickle('pnp_data')
df.head()

,claim_id,customer_id,customer_name,relationship_start_date,country,gender,postcode,employment_status,age,account_type,first_claim,total_claims_in_90days,no_of_trans,trans_id,trans_datetime,trans_amount,claim_amount,claim_vol
0,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711005,2022-04-23,150,2333,5
1,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711017,2022-01-09,833,2333,5
2,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711040,2022-03-25,39,2333,5
3,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711057,2022-04-14,610,2333,5
4,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711067,2022-01-31,701,2333,5


In [101]:
#df = fake_df
#df.head()

#### Adding Extra Layer to the Data

In [102]:
##This is required for our data to be consistent.
df['total_claims_in_90days'] = np.where(
    df['first_claim'] == 'Yes', 0, np.where(
    ((df['first_claim'] == 'No') & (df['total_claims_in_90days']== 0)), 1 ,df['total_claims_in_90days']))

In [103]:
sqldf("select first_claim,total_claims_in_90days, count(*) as tot from df group by first_claim,total_claims_in_90days")

,first_claim,total_claims_in_90days,tot
0,No,1,3
1,No,2,1
2,No,3,5
3,No,4,2
4,No,6,8
5,No,7,4
6,No,8,3
7,No,9,10
8,Yes,0,34


Data Assumptions
1. claim_id can be duplicated as a claim can have more than 1 transaction on it. Meaning a customer can file claim for multiple transactions on just one claim application.
2. Customers needs to be operating the account for at least a year for it to be included in this analysis.(This may not hold in real life scenario).
3. Age has been pegged between 18 to 95 (Children accounts excluded)
4. If First Claim = 'Yes' then we would expect to have Total Claims in the last 90 days to be 0.

#### Data Manipulation

#### Calculating Date Difference

In [104]:
##df['relationship_start_date'] = df['relationship_start_date'].astype('datetime64[ns]')
df['relationship_start_date'] = pd.to_datetime(df['relationship_start_date'])

In [105]:
df['time_with_bank'] = abs((df['relationship_start_date'] - pd.to_datetime("now")).dt.days)
df['time_with_bank']

df.head()

,claim_id,customer_id,customer_name,relationship_start_date,country,gender,postcode,employment_status,age,account_type,first_claim,total_claims_in_90days,no_of_trans,trans_id,trans_datetime,trans_amount,claim_amount,claim_vol,time_with_bank
0,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711005,2022-04-23,150,2333,5,8664
1,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711017,2022-01-09,833,2333,5,8664
2,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711040,2022-03-25,39,2333,5,8664
3,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711057,2022-04-14,610,2333,5,8664
4,500000001,119001,Courtney Howell,1998-09-15,Macedonia,Male,W2T 1WR,Full Time,82,Savings,No,9,7,5180711067,2022-01-31,701,2333,5,8664


#### 1. Which age brackets have got previous claim of more than 4?

In [107]:
def age_group(age):
    if age >= 18 and age < 25:
        age_bracket = '18 - 24'
    elif age >= 25  and age < 45 :
        age_bracket = '25 - 44'
    elif age >= 45  and age < 65 :
        age_bracket = '45 - 64'
    else:
        age_bracket = 'above 64'
    return age_bracket

In [111]:
#Note: I could have also used case statement in SQL to achieve this step.
df['age_group'] = df.apply(lambda x: age_group(x['age']), axis=1)
t = sqldf("select distinct customer_id,count(distinct customer_id) as count, age_group,total_claims_in_90days from df where total_claims_in_90days > 4 group by customer_id, age_group order by age_group, total_claims_in_90days desc")
t_0 = sqldf("select age_group,sum(count) as counter,sum(total_claims_in_90days) as total_in_90_days from t group by age_group")
t_0['percent'] = t_0['total_in_90_days']/t_0['total_in_90_days'].sum()
t_0['percent'] = t_0['percent'].map('{:.2%}'.format)
t_0

,age_group,counter,total_in_90_days,percent
0,25 - 44,1,9,20.00%
1,45 - 64,2,15,33.33%
2,above 64,3,21,46.67%


In [112]:
sqldf("select count(distinct customer_id) as tot from df ")

,tot
0,20


In [113]:
##Data Validation

In [114]:
df[(df['total_claims_in_90days']>4) & (df['age_group']=='25 - 44')]

,claim_id,customer_id,customer_name,relationship_start_date,country,gender,postcode,employment_status,age,account_type,first_claim,total_claims_in_90days,no_of_trans,trans_id,trans_datetime,trans_amount,claim_amount,claim_vol,time_with_bank,age_group
10,500000003,119003,Justin Hamilton,2020-10-25,Kenya,Female,SL0 6AW,Per Diem,37,Current,No,9,4,5180711037,2022-04-13,85,1472,5,588,25 - 44
11,500000003,119003,Justin Hamilton,2020-10-25,Kenya,Female,SL0 6AW,Per Diem,37,Current,No,9,4,5180711044,2022-02-01,453,1472,5,588,25 - 44
12,500000003,119003,Justin Hamilton,2020-10-25,Kenya,Female,SL0 6AW,Per Diem,37,Current,No,9,4,5180711049,2022-02-23,143,1472,5,588,25 - 44
13,500000003,119003,Justin Hamilton,2020-10-25,Kenya,Female,SL0 6AW,Per Diem,37,Current,No,9,4,5180711055,2022-04-02,591,1472,5,588,25 - 44
14,500000003,119003,Justin Hamilton,2020-10-25,Kenya,Female,SL0 6AW,Per Diem,37,Current,No,9,4,5180711056,2022-01-21,200,1472,5,588,25 - 44


#### 2. Which class of customer have got volume of claims above 3?

In [115]:
sqldf("select distinct customer_id,gender,age,account_type, age_group,claim_vol from df where claim_vol > 3 order by age_group,account_type,claim_vol ")

,customer_id,gender,age,account_type,age_group,claim_vol
0,119003,Female,37,Current,25 - 44,5
1,119008,Male,43,Savings,25 - 44,4
2,119014,Male,52,Current,45 - 64,4
3,119019,Male,58,Current,45 - 64,5
4,119006,Female,74,Current,above 64,4
5,119002,Male,94,Current,above 64,5
6,119015,Female,84,Current,above 64,6
7,119001,Male,82,Savings,above 64,5
8,119012,Male,65,Savings,above 64,7


#### 3. What is the average number of days for a customer to be with the bank for it to file a claim?

In [116]:
sqldf("select first_claim, count(first_claim) as total,round(avg(time_with_bank),0) as mean_time_in_days from (select distinct customer_id,first_claim,time_with_bank from df) group by first_claim")

,first_claim,total,mean_time_in_days
0,No,10,4662.0
1,Yes,10,5263.0


#### 4. Who filed claim the more - Men or Women?

In [117]:
sqldf("select gender, sum(claim_vol) as tot_claim from (select distinct customer_id,gender,claim_vol from df) group by gender")

,gender,tot_claim
0,Female,19
1,Male,51


#### 5. Which account type gets more than 3 claims for customers between 20 and 55 years old and having joined the bank more than 2 years?

In [16]:
sqldf("select account_type,age_group, count(*) as Total \
      from df where claim_vol > 3 and time_with_bank > 2*360  and age between 20 and 55 \
        group by account_type,age_group")

,account_type,age_group,Total
0,Current,18 - 24,6
1,Current,25 - 44,8


In [22]:
sqldf("select account_type, count(distinct trans_id) as tot, sum(trans_amount) as vol from df group by account_type ")

,account_type,tot,vol
0,Current,48,25054
1,Savings,22,10541


In [69]:
sqldf("select * from df where trans_amount < 100")

,claim_id,customer_id,customer_name,relationship_start_date,country,gender,postcode,employment_status,age,account_type,first_claim,total_claims_in_90days,no_of_trans,trans_id,trans_datetime,trans_amount,claim_amount,claim_vol,age_group
0,500000001,119001,Erin Rice,2020-10-21,Norway,Male,IP5 5WE,Full Time,51,Current,Yes,0,13,5180711008,2022-02-28,27,2040,4,45 - 64
1,500000005,119005,Michelle King,2010-02-20,Argentina,Male,26315,Part Time,80,Savings,Yes,0,11,5180711012,2022-04-06,61,1225,3,above 64
2,500000010,119010,Mary Kramer,2011-09-28,China,Male,54899,Full Time,62,Savings,Yes,0,7,5180711054,2022-05-02,10,1734,6,45 - 64


#### The Business RULE to be coded

In [90]:
#You've got to wrap each of the conditions in brackets for it to work.

conditions = [
    (df['claim_amount'] >= 100),
    ((df['time_with_bank'] >= 365) & (df['no_of_trans'] >= 3)),
    ((df['employment_status'] != 'Full Time') & (df['gender'] == 'Male') & ((df['age'] >= 25) & (df['age'] <= 45))),
    ((df['first_claim'] != 'No') & (df['total_claims_in_90days'] >= 2))
]

choices = ['Referred','Referred','Referred','Referred']

df['decision'] = np.select(conditions, choices, default= 'Auto Pay')

df.head()

,claim_id,customer_id,customer_name,relationship_start_date,country,gender,postcode,employment_status,age,account_type,...,total_claims_in_90days,no_of_trans,trans_id,trans_datetime,trans_amount,claim_amount,claim_vol,time_with_bank,age_group,decision
0,500000001,119001,Jonathan Jones,2008-08-27,Bulgaria,Male,42512,Resigned,50,Savings,...,0,13,5180711011,2022-03-28,743,3261,6,5030,45 - 64,Referred
1,500000001,119001,Jonathan Jones,2008-08-27,Bulgaria,Male,42512,Resigned,50,Savings,...,0,13,5180711019,2022-02-19,602,3261,6,5030,45 - 64,Referred
2,500000001,119001,Jonathan Jones,2008-08-27,Bulgaria,Male,42512,Resigned,50,Savings,...,0,13,5180711046,2022-05-24,542,3261,6,5030,45 - 64,Referred
3,500000001,119001,Jonathan Jones,2008-08-27,Bulgaria,Male,42512,Resigned,50,Savings,...,0,13,5180711051,2022-03-14,274,3261,6,5030,45 - 64,Referred
4,500000001,119001,Jonathan Jones,2008-08-27,Bulgaria,Male,42512,Resigned,50,Savings,...,0,13,5180711054,2022-02-23,355,3261,6,5030,45 - 64,Referred


In [91]:
len(df[df['decision']=='Referred'])

70